# Building Machine Learning Classifiers: Evaluate Gradient Boosting with GridSearchCV

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

**Cross-validation:** Divide a dataset into k subsets and repeat the holdout method k times where a different subset is used as the holdout set in each iteration.

### Read in text

In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)

# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)

X_count_feat.head()

body_len  punct%  0  1  2  3  4  5  6  7  ...   8094  8095  8096  8097  \
0       128     4.7  0  0  0  0  0  0  0  0  ...      0     0     0     0   
1        49     4.1  0  0  0  0  0  0  0  0  ...      0     0     0     0   
2        62     3.2  0  0  0  0  0  0  0  0  ...      0     0     0     0   
3        28     7.1  0  0  0  0  0  0  0  0  ...      0     0     0     0   
4       135     4.4  0  0  0  0  0  0  0  0  ...      0     0     0     0   

   8098  8099  8100  8101  8102  8103  
0     0     0     0     0     0     0  
1     0     0     0     0     0     0  
2     0     0     0     0     0     0  
3     0     0     0     0     0     0  
4     0     0     0     0     0     0  

[5 rows x 8106 columns]

### Exploring parameter settings using GridSearchCV

In [2]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

In [5]:
gb = GradientBoostingClassifier()
param = {
    'n_estimators': [100, 150], 
    'max_depth': [7, 11, 15],
    'learning_rate': [0.1]
}

clf = GridSearchCV(gb, param, cv=5, n_jobs=-1)
cv_fit = clf.fit(X_tfidf_feat, data['label'])
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

mean_fit_time  mean_score_time  mean_test_score  mean_train_score  \
5     368.558762         0.308644         0.969643               1.0   
2     214.185205         0.303001         0.969283               1.0   
8     381.475534         0.206334         0.968744               1.0   
1     157.876316         0.351555         0.968205               1.0   
7     353.960683         0.258787         0.968205               1.0   

  param_learning_rate param_max_depth param_n_estimators  \
5                 0.1              11                150   
2                 0.1               7                150   
8                 0.1              15                150   
1                 0.1               7                100   
7                 0.1              15                100   

                                              params  rank_test_score  \
5  {'n_estimators': 150, 'learning_rate': 0.1, 'm...                1   
2  {'n_estimators': 150, 'learning_rate': 0.1, 'm...                2   
8  {'n_estimators': 150, 'learning_rate': 0.1, 'm...                3   
1  {'n_estimators': 100, 'learning_rate': 0.1, 'm...                4   
7  {'n_estimators': 100, 'learning_rate': 0.1, 'm...                4   

   split0_test_score       ...         split2_test_score  split2_train_score  \
5           0.965919       ...                  0.966757                 1.0   
2           0.965919       ...                  0.968553                 1.0   
8           0.965022       ...                  0.969452                 1.0   
1           0.965919       ...                  0.968553                 1.0   
7           0.964126       ...                  0.969452                 1.0   

   split3_test_score  split3_train_score  split4_test_score  \
5           0.968553                 1.0           0.969452   
2           0.964960                 1.0           0.967655   
8           0.965858                 1.0           0.967655   
1           0.964960                 1.0           0.965858   
7           0.966757                 1.0           0.967655   

   split4_train_score  std_fit_time  std_score_time  std_test_score  \
5                 1.0      7.128055        0.030754        0.004141   
2                 1.0      2.241176        0.043978        0.005181   
8                 1.0     44.316129        0.036329        0.003816   
1                 1.0      3.158135        0.187909        0.003954   
7                 1.0     10.086312        0.056744        0.002968   

   std_train_score  
5              0.0  
2              0.0  
8              0.0  
1              0.0  
7              0.0  

[5 rows x 23 columns]

In [7]:
gb = GradientBoostingClassifier()
param = {
    'n_estimators': [50, 100, 150], 
    'max_depth': [7, 11, 15],
    'learning_rate': [0.1]
}

clf = GridSearchCV(gb, param, cv=5, n_jobs=-1)
cv_fit = clf.fit(X_count_feat, data['label'])
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

mean_fit_time  mean_score_time  mean_test_score  mean_train_score  \
5     341.866984         0.335839         0.969463          1.000000   
7     335.012637         0.264707         0.969283          1.000000   
2     213.266939         0.279388         0.968385          0.999955   
8     356.912155         0.183561         0.968385          1.000000   
4     241.101819         0.263556         0.968205          1.000000   

  param_learning_rate param_max_depth param_n_estimators  \
5                 0.1              11                150   
7                 0.1              15                100   
2                 0.1               7                150   
8                 0.1              15                150   
4                 0.1              11                100   

                                              params  rank_test_score  \
5  {'n_estimators': 150, 'learning_rate': 0.1, 'm...                1   
7  {'n_estimators': 100, 'learning_rate': 0.1, 'm...                2   
2  {'n_estimators': 150, 'learning_rate': 0.1, 'm...                3   
8  {'n_estimators': 150, 'learning_rate': 0.1, 'm...                3   
4  {'n_estimators': 100, 'learning_rate': 0.1, 'm...                5   

   split0_test_score       ...         split2_test_score  split2_train_score  \
5           0.965022       ...                  0.970350                 1.0   
7           0.965022       ...                  0.967655                 1.0   
2           0.965919       ...                  0.968553                 1.0   
8           0.962332       ...                  0.967655                 1.0   
4           0.963229       ...                  0.970350                 1.0   

   split3_test_score  split3_train_score  split4_test_score  \
5           0.963163                 1.0           0.970350   
7           0.964960                 1.0           0.972147   
2           0.960467                 1.0           0.966757   
8           0.963163                 1.0           0.971249   
4           0.961366                 1.0           0.968553   

   split4_train_score  std_fit_time  std_score_time  std_test_score  \
5            1.000000      3.450213        0.048011        0.005320   
7            1.000000      7.205083        0.027251        0.004513   
2            0.999775      2.651567        0.022217        0.006508   
8            1.000000     45.305546        0.031481        0.005594   
4            1.000000      3.736889        0.026132        0.005716   

   std_train_score  
5          0.00000  
7          0.00000  
2          0.00009  
8          0.00000  
4          0.00000  

[5 rows x 23 columns]